## for AWS Exam Discusstion Extract

In [3]:
import time
import re
import os
import winsound
import pandas as pd

import pyautogui

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup

from random import *
from pathlib import Path
from dateutil import parser

from datetime import datetime
import time

In [28]:
def set_chrome_driver():
    service = Service(executable_path=r'c:/temp/chromedriver.exe')
    options = webdriver.ChromeOptions()
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    options.add_argument('--no-sandbox')
    options.add_argument('--ignore-certificate-errors')
    driver = webdriver.Chrome(service=service, options=options)

    return driver


In [71]:

def set_headless_mode(driver, headless):
    driver.quit()
    service = Service(executable_path=r'c:/temp/chromedriver.exe')
    options = Options()
    if headless:
        options.add_argument("--headless")
    else:
        # Copy the existing arguments except for the "--headless" argument
        for arg in driver.options.arguments:
            if arg != "--headless":
                options.add_argument(arg)
    return webdriver.Chrome(service=service, options=options)

In [72]:
driver = set_headless_mode(driver, True)

In [73]:
    driver = set_chrome_driver()

In [75]:
    print(driver.title + "-" + str(1))
    bs = BeautifulSoup(driver.page_source, 'html.parser')
    div_card = bs.find_all("div", {"class": "container-fluid discussion-list"})
    a = bs.find_all("a", {"class": "discussion-link"})
    span = bs.find_all("span", {"class": "recent-post-time"})
    
    found = False
    for i in range(len(a)):
        if found == True: break
        
        last_post = parser.parse(str(span[i*2+1]["title"]).replace("midnight", "12:00 a.m.").replace("noon", "12:00 p.m."))
        split_text = str(a[i].text.strip()).split()

        question = ' '.join(split_text[:5])
        number = split_text[5]
        discussion = split_text[6]
        did = str(a[i]["href"]).split('/')[4].split('-')[0]

        print(question+"\t"+str(number)+"\t"+discussion+"\t"+str(last_post)+"\t"+str(a[i]["href"])+"\t"+did+"\n")
        if (df['ExamType'] == question).any() and \
            (df['ExamNo'] == number).any() and \
            (df['DiscussNo'] == did).any():
            print("The values exist in the DataFrame.")
            if (df['LastPost'] == str(last_post)).any():
                print("Same value exists, exit")
                found = True
                break
            print(df.loc[(df['ExamType'] == question) & (df['ExamNo'] == number) & (df['DiscussNo'] == did), 'LastPost'], str(last_post))
            df.loc[(df['ExamType'] == question) & (df['ExamNo'] == number) & (df['DiscussNo'] == did), 'LastPost'] = str(last_post)
        else:
            print("The values do not exist in the DataFrame.")
            new_row = [{ 'ExamType': question, 'ExamNo': number, 'Discussion': discussion, 'LastPost': str(last_post),
                         'DiscussURL': a[i]["href"], 'DiscussNo': did }]
            df = pd.concat([df, pd.DataFrame(new_row)], ignore_index=True)


-1


## DataFrame Test

In [7]:
import pandas as pd

df = pd.read_csv('IsacaDiscuss.txt', delimiter='\t', encoding='utf-8', header=None,
                 names=['ExamType', 'ExamNo', 'DiscussNo', 'DataID', 'LastPost', 'DiscussURL'], index_col=False)
                 #names=['ExamType', 'ExamNo', 'DiscussNo', 'LastPost', 'DiscussURL'], index_col=False)
# df = df[df['ExamType'] == 'Exam AWS Certified Solutions Architect - Professional topic 1']
df

,ExamType,ExamNo,DiscussNo,DataID,LastPost,DiscussURL
0,Exam CRISC topic 1,642,26685,0,2023-09-01 23:52:00,/discussions/isaca/view/26685-exam-crisc-topic...
1,Exam CRISC topic 1,641,47984,0,2023-09-01 23:42:00,/discussions/isaca/view/47984-exam-crisc-topic...
2,Exam CRISC topic 1,640,54258,0,2023-09-01 23:36:00,/discussions/isaca/view/54258-exam-crisc-topic...
3,Exam CRISC topic 1,639,30411,0,2023-09-01 23:26:00,/discussions/isaca/view/30411-exam-crisc-topic...
4,Exam CRISC topic 1,638,30167,0,2023-09-01 23:24:00,/discussions/isaca/view/30167-exam-crisc-topic...
...,...,...,...,...,...,...
5446,Exam CISM topic 1,327,28674,785137,2020-09-12 14:23:00,/discussions/isaca/view/28674-exam-cism-topic-...
5447,Exam CISA topic 1,775,30067,750624,2020-09-12 07:11:00,/discussions/isaca/view/30067-exam-cisa-topic-...
5448,Exam CISA topic 1,288,30880,750137,2020-09-08 20:37:00,/discussions/isaca/view/30880-exam-cisa-topic-...
5449,Exam CISA topic 1,827,28815,750676,2020-09-03 15:05:00,/discussions/isaca/view/28815-exam-cisa-topic-...


In [8]:

# Convert the "Date" column to datetime format
df['ExamType'] = df['ExamType'].str.strip()
df['ExamNo'] = df['ExamNo'].astype(int)
df['DiscussNo'] = df['DiscussNo'].astype(int)
df['DataID'] = df['DataID'].astype(int)
df['LastPost'] = pd.to_datetime(df['LastPost'], format='%Y-%m-%d %H:%M')

df.drop_duplicates(inplace=True)
df = df.sort_values(['LastPost', 'DiscussNo'], ascending=[False, False])
df

,ExamType,ExamNo,DiscussNo,DataID,LastPost,DiscussURL
0,Exam CRISC topic 1,642,26685,0,2023-09-01 23:52:00,/discussions/isaca/view/26685-exam-crisc-topic...
1,Exam CRISC topic 1,641,47984,0,2023-09-01 23:42:00,/discussions/isaca/view/47984-exam-crisc-topic...
2,Exam CRISC topic 1,640,54258,0,2023-09-01 23:36:00,/discussions/isaca/view/54258-exam-crisc-topic...
3,Exam CRISC topic 1,639,30411,0,2023-09-01 23:26:00,/discussions/isaca/view/30411-exam-crisc-topic...
4,Exam CRISC topic 1,638,30167,0,2023-09-01 23:24:00,/discussions/isaca/view/30167-exam-crisc-topic...
...,...,...,...,...,...,...
4452,Exam CRISC topic 4,35,13273,0,2020-02-02 13:42:00,/discussions/isaca/view/13273-exam-crisc-topic...
4453,Exam CISA topic 2,4,7814,342141,2020-01-18 08:13:00,/discussions/isaca/view/7814-exam-cisa-topic-2...
4454,Exam CISA topic 1,3208,7827,753057,2020-01-15 22:59:00,/discussions/isaca/view/7827-exam-cisa-topic-1...
4455,Exam COBIT 5 topic 1,25,1941,0,2019-06-21 21:46:00,/discussions/isaca/view/1941-exam-cobit-5-topi...


In [9]:
# Update 'PostDate' to the maximum value within each group of 'Type' and 'No'
df['MaxDataID'] = df.groupby(['ExamType', 'ExamNo'])['DataID'].transform('max')
df['Chk'] = df.apply(lambda row: 1 if row['DataID'] == row['MaxDataID'] else 0, axis=1)
df = df.sort_values(['Chk', 'LastPost', 'DiscussNo'], ascending=[False, False, False]).drop(columns=['MaxDataID', 'Chk'])
# Save the modified DataFrame back to a CSV file
df.to_csv('modified_mydata.csv', index=False, sep='\t', encoding='utf-8', header=None)
df

,ExamType,ExamNo,DiscussNo,DataID,LastPost,DiscussURL
0,Exam CRISC topic 1,642,26685,0,2023-09-01 23:52:00,/discussions/isaca/view/26685-exam-crisc-topic...
1,Exam CRISC topic 1,641,47984,0,2023-09-01 23:42:00,/discussions/isaca/view/47984-exam-crisc-topic...
2,Exam CRISC topic 1,640,54258,0,2023-09-01 23:36:00,/discussions/isaca/view/54258-exam-crisc-topic...
3,Exam CRISC topic 1,639,30411,0,2023-09-01 23:26:00,/discussions/isaca/view/30411-exam-crisc-topic...
4,Exam CRISC topic 1,638,30167,0,2023-09-01 23:24:00,/discussions/isaca/view/30167-exam-crisc-topic...
...,...,...,...,...,...,...
5446,Exam CISM topic 1,327,28674,785137,2020-09-12 14:23:00,/discussions/isaca/view/28674-exam-cism-topic-...
5447,Exam CISA topic 1,775,30067,750624,2020-09-12 07:11:00,/discussions/isaca/view/30067-exam-cisa-topic-...
5448,Exam CISA topic 1,288,30880,750137,2020-09-08 20:37:00,/discussions/isaca/view/30880-exam-cisa-topic-...
5449,Exam CISA topic 1,827,28815,750676,2020-09-03 15:05:00,/discussions/isaca/view/28815-exam-cisa-topic-...


In [36]:
def sleep_random_sec(sec):
    for t in reversed(range(randint(1, sec))):
        print(str(t), end='', flush=True)
        time.sleep(1)
    print('')
    
def open_exam(driver, discuss_url):
    sleep_random_sec(3)
    exam_url = 'https://www.examtopics.com' + discuss_url
    pattern = r'<div class="question-body mt-3 pt-3 border-top" data-id="([0-9]+)">'

    page = driver.get(exam_url)

    bs = BeautifulSoup(driver.page_source, 'html.parser')
    div_discuss = bs.find_all("div", {"class": "container outer-discussion-container"})
    while (len(div_discuss) <= 0):
        print("Waiting...")
        bs = BeautifulSoup(driver.page_source, 'html.parser')
        div_discuss = bs.find_all("div", {"class": "container outer-discussion-container"})
        if (driver.title == 'Error 404 (Not Found)!!1'):
            break
        winsound.PlaySound("SystemExclamation", winsound.SND_ALIAS)
        time.sleep(3)
    
    match = re.search(pattern, str(bs))
    if match:
        print(f'url={discuss_url}, data_id = {match.group(1)}')

    return



In [37]:
urls = [
   { 'url': '/discussions/amazon/view/24902-exam-aws-certified-solutions-architect-associate-saa-c02/' },
   { 'url': '/discussions/amazon/view/27940-exam-aws-certified-solutions-architect-associate-saa-c02/' },
   { 'url': '/discussions/amazon/view/21792-exam-aws-certified-solutions-architect-associate-saa-c02/' },
   { 'url': '/discussions/amazon/view/24833-exam-aws-certified-solutions-architect-associate-saa-c02/' },
   { 'url': '/discussions/amazon/view/21962-exam-aws-certified-solutions-architect-associate-saa-c02/' },
   { 'url': '/discussions/amazon/view/21964-exam-aws-certified-solutions-architect-associate-saa-c02/' },
   { 'url': '/discussions/amazon/view/21830-exam-aws-certified-solutions-architect-associate-saa-c02/' },
   { 'url': '/discussions/amazon/view/27839-exam-aws-certified-solutions-architect-associate-saa-c02/' },
   { 'url': '/discussions/amazon/view/27840-exam-aws-certified-solutions-architect-associate-saa-c02/' },
   { 'url': '/discussions/amazon/view/27837-exam-aws-certified-solutions-architect-associate-saa-c02/' },
   { 'url': '/discussions/amazon/view/27838-exam-aws-certified-solutions-architect-associate-saa-c02/' },
   { 'url': '/discussions/amazon/view/27836-exam-aws-certified-solutions-architect-associate-saa-c02/' },
   { 'url': '/discussions/amazon/view/46358-exam-aws-certified-solutions-architect-associate-saa-c02/' },
   { 'url': '/discussions/amazon/view/25147-exam-aws-certified-database-specialty-topic-1-question-20/' },
   { 'url': '/discussions/amazon/view/22182-exam-aws-certified-solutions-architect-associate-saa-c02/' },
   { 'url': '/discussions/amazon/view/28085-exam-aws-certified-solutions-architect-associate-saa-c02/' },
   { 'url': '/discussions/amazon/view/5769-exam-aws-certified-solutions-architect-professional-topic-1/' },
   { 'url': '/discussions/amazon/view/51220-exam-aws-certified-solutions-architect-professional-topic-1/' },
   { 'url': '/discussions/amazon/view/60090-exam-aws-certified-solutions-architect-professional-topic-1/' },
   { 'url': '/discussions/amazon/view/29771-exam-aws-certified-solutions-architect-associate-saa-c02/' },
   { 'url': '/discussions/amazon/view/24825-exam-aws-certified-solutions-architect-associate-saa-c02/' },
   { 'url': '/discussions/amazon/view/46600-exam-aws-certified-solutions-architect-associate-saa-c02/' },
   { 'url': '/discussions/amazon/view/74125-exam-aws-certified-solutions-architect-professional-topic-1/' },
   { 'url': '/discussions/amazon/view/61148-exam-aws-certified-solutions-architect-associate-saa-c02/' },
   { 'url': '/discussions/amazon/view/48842-exam-aws-certified-database-specialty-topic-1-question-141/' },
   { 'url': '/discussions/amazon/view/36057-exam-aws-certified-solutions-architect-professional-topic-1/' },
   { 'url': '/discussions/amazon/view/59990-exam-aws-certified-solutions-architect-professional-topic-1/' },
   { 'url': '/discussions/amazon/view/46368-exam-aws-certified-solutions-architect-associate-saa-c02/' },
   { 'url': '/discussions/amazon/view/5176-exam-aws-certified-solutions-architect-professional-topic-1/' },
   { 'url': '/discussions/amazon/view/13316-exam-aws-certified-solutions-architect-professional-topic-1/' },
   { 'url': '/discussions/amazon/view/67692-exam-aws-certified-solutions-architect-associate-saa-c02/' },
   { 'url': '/discussions/amazon/view/67656-exam-aws-certified-solutions-architect-associate-saa-c02/' },
   { 'url': '/discussions/amazon/view/46963-exam-aws-certified-solutions-architect-professional-topic-1/' },
   { 'url': '/discussions/amazon/view/46832-exam-aws-certified-solutions-architect-professional-topic-1/' },
   { 'url': '/discussions/amazon/view/26323-exam-aws-certified-database-specialty-topic-1-question-56/' },
   { 'url': '/discussions/amazon/view/25815-exam-aws-certified-database-specialty-topic-1-question-31/' },
   { 'url': '/discussions/amazon/view/61159-exam-aws-certified-solutions-architect-associate-saa-c02/' },
   { 'url': '/discussions/amazon/view/25196-exam-aws-certified-database-specialty-topic-1-question-49/' },
   { 'url': '/discussions/amazon/view/25869-exam-aws-certified-database-specialty-topic-1-question-2/' },
   { 'url': '/discussions/amazon/view/26424-exam-aws-certified-database-specialty-topic-1-question-52/' },
   { 'url': '/discussions/amazon/view/25068-exam-aws-certified-database-specialty-topic-1-question-14/' },
   { 'url': '/discussions/amazon/view/26021-exam-aws-certified-database-specialty-topic-1-question-44/' },
   { 'url': '/discussions/amazon/view/5127-exam-aws-certified-solutions-architect-professional-topic-1/' },
   { 'url': '/discussions/amazon/view/26592-exam-aws-certified-database-specialty-topic-1-question-63/' },
   { 'url': '/discussions/amazon/view/26231-exam-aws-certified-database-specialty-topic-1-question-29/' },
   { 'url': '/discussions/amazon/view/12853-exam-aws-certified-solutions-architect-professional-topic-1/' },
   { 'url': '/discussions/amazon/view/8764-exam-aws-certified-solutions-architect-professional-topic-1/' },
   { 'url': '/discussions/amazon/view/7627-exam-aws-certified-solutions-architect-professional-topic-1/' },
   { 'url': '/discussions/amazon/view/5126-exam-aws-certified-solutions-architect-professional-topic-1/' },
   { 'url': '/discussions/amazon/view/25620-exam-aws-certified-database-specialty-topic-1-question-46/' },
   { 'url': '/discussions/amazon/view/26803-exam-aws-certified-database-specialty-topic-1-question-53/' },
   { 'url': '/discussions/amazon/view/25599-exam-aws-certified-database-specialty-topic-1-question-32/' },
   { 'url': '/discussions/amazon/view/26774-exam-aws-certified-database-specialty-topic-1-question-61/' },
   { 'url': '/discussions/amazon/view/25725-exam-aws-certified-database-specialty-topic-1-question-39/' },
   { 'url': '/discussions/amazon/view/31126-exam-aws-certified-solutions-architect-professional-topic-1/' },
   { 'url': '/discussions/amazon/view/25639-exam-aws-certified-database-specialty-topic-1-question-16/' },
   { 'url': '/discussions/amazon/view/5210-exam-aws-certified-solutions-architect-professional-topic-1/' },
   { 'url': '/discussions/amazon/view/11802-exam-aws-certified-solutions-architect-professional-topic-1/' },
   { 'url': '/discussions/amazon/view/28991-exam-aws-certified-solutions-architect-professional-topic-1/' },
   { 'url': '/discussions/amazon/view/11453-exam-aws-certified-solutions-architect-professional-topic-1/' },
   { 'url': '/discussions/amazon/view/25733-exam-aws-certified-database-specialty-topic-1-question-8/' },
   { 'url': '/discussions/amazon/view/12300-exam-aws-certified-solutions-architect-professional-topic-1/' },
   { 'url': '/discussions/amazon/view/25203-exam-aws-certified-database-specialty-topic-1-question-60/' },
   { 'url': '/discussions/amazon/view/9895-exam-aws-certified-solutions-architect-professional-topic-1/' },
   { 'url': '/discussions/amazon/view/19716-exam-aws-certified-solutions-architect-professional-topic-1/' },
   { 'url': '/discussions/amazon/view/31264-exam-aws-certified-solutions-architect-professional-topic-1/' },
   { 'url': '/discussions/amazon/view/12325-exam-aws-certified-solutions-architect-professional-topic-1/' },
   { 'url': '/discussions/amazon/view/14891-exam-aws-certified-solutions-architect-professional-topic-1/' },
   { 'url': '/discussions/amazon/view/53233-exam-aws-certified-solutions-architect-professional-topic-1/' },
   { 'url': '/discussions/amazon/view/12297-exam-aws-certified-solutions-architect-professional-topic-1/' },
]

driver = set_chrome_driver()

for i in range(len(urls)):
   #print(urls[i]['url'])
   open_exam(driver, urls[i]['url'])

driver.quit()


0
url=/discussions/amazon/view/24902-exam-aws-certified-solutions-architect-associate-saa-c02/, data_id = 557466
210
url=/discussions/amazon/view/27940-exam-aws-certified-solutions-architect-associate-saa-c02/, data_id = 557401
10
url=/discussions/amazon/view/21792-exam-aws-certified-solutions-architect-associate-saa-c02/, data_id = 553112
10
url=/discussions/amazon/view/24833-exam-aws-certified-solutions-architect-associate-saa-c02/, data_id = 557465
210
url=/discussions/amazon/view/21962-exam-aws-certified-solutions-architect-associate-saa-c02/, data_id = 553092
0
url=/discussions/amazon/view/21964-exam-aws-certified-solutions-architect-associate-saa-c02/, data_id = 553111
0
url=/discussions/amazon/view/21830-exam-aws-certified-solutions-architect-associate-saa-c02/, data_id = 553134
10
url=/discussions/amazon/view/27839-exam-aws-certified-solutions-architect-associate-saa-c02/, data_id = 557447
0
url=/discussions/amazon/view/27840-exam-aws-certified-solutions-architect-associate-saa

In [ ]:
def open_page(driver, url):
    discuss_url = 'https://www.examtopics.com' + str(url) + '/'
    # discuss_url = 'https://www.examtopics.com/ajax/discussion/load-complete/?discussion-id=35981'
    page = driver.get(discuss_url)
    time.sleep(randint(1, 5))
    return page


In [ ]:
# Define formatting function
def format_column(column, length, column_type):
    if column_type == 'string':
        return column.ljust(length)  # Left-align string columns
    elif column_type == 'int':
        return str(column).rjust(length)  # Right-align integer columns
    else:
        return column

# Define column lengths and types
column_lengths = [70, 5, 8, 20, 120]
column_types = ['string', 'int', 'int', 'string', 'string']

# Open the file in write mode
filename = 'output.txt'
with open(filename, 'w') as file:
    # Write data rows
    for _, row in df.iterrows():
        formatted_row = [format_column(row[col], length, col_type) for col, length, col_type in zip(df.columns, column_lengths, column_types)]
        file.write('\t'.join(formatted_row) + '\n')

: 

In [ ]:
    driver.quit()

In [ ]:
def open_isacadiscuss(driver, pageno):
    discuss_url = 'https://www.examtopics.com/discussions/isaca/' + str(pageno) + '/'
    # discuss_url = 'https://www.examtopics.com/ajax/discussion/load-complete/?discussion-id=35981'
    page = driver.get(discuss_url)
    time.sleep(randint(1, 5))
    return


In [ ]:
if __name__ == "__main__":
    driver = set_chrome_driver()

    driver.set_window_position(1800,10)
    
    with open("IsacaDiscussList.txt", "a", encoding='utf-8') as file:    
        for i in range(254)[:2]:
            pageno = i + 1

            open_isacadiscuss(driver, pageno)

            driver.switch_to.window(driver.window_handles[0])
            print(driver.title + "-" + str(pageno))
            bs = BeautifulSoup(driver.page_source, 'html.parser')
            div_card = bs.find_all("div", {"class": "container-fluid discussion-list"})
            a = bs.find_all("a", {"class": "discussion-link"})
            span = bs.find_all("span", {"class": "recent-post-time"})
            for i in range(len(a)):
                last_post = parser.parse(str(span[i*2+1]["title"]).replace("midnight", "12:00 a.m.").replace("noon", "12:00 p.m."))
                split_text = str(a[i].text.strip()).split()

                question = ' '.join(split_text[:5])
                number = split_text[5]
                discussion = split_text[6]
                did = str(a[i]["href"]).split('/')[4].split('-')[0]
                
                
                file.write(question+"\t"+str(number)+"\t"+discussion+"\t"+str(last_post)+"\t"+str(a[i]["href"])+"\t"+did+"\n")                   

    file.close()
    
    driver.quit()
    

Isaca discussions - ExamTopics Forum-1
Isaca discussions - ExamTopics Forum-2


In [ ]:
    pageno = 1
    print(driver.title + " - " + str(pageno))

    bs = BeautifulSoup(driver.page_source, 'html.parser')
    div_card = bs.find_all("div", {"class": "container-fluid discussion-list"})
    a = bs.find_all("a", {"class": "discussion-link"})
    span = bs.find_all("span", {"class": "recent-post-time"})
    for i in range(len(a)):
        last_post = parser.parse(str(span[i*2+1]["title"]))
        print(str(a[i].text.strip())+"\t"+str(last_post)+"\t"+str(a[i]["href"])+"\n")
        #print(str(last_post)+"\t")
        #print(str(a[i]["href"])+"\n")
    

Isaca discussions - ExamTopics Forum - 1
Exam CISM topic 1 question 820 discussion	2023-07-08 20:40:00	/discussions/isaca/view/114534-exam-cism-topic-1-question-820-discussion/

Exam CISM topic 1 question 451 discussion	2023-07-08 19:51:00	/discussions/isaca/view/113467-exam-cism-topic-1-question-451-discussion/

Exam CISM topic 1 question 337 discussion	2023-07-08 18:11:00	/discussions/isaca/view/94539-exam-cism-topic-1-question-337-discussion/

Exam CISA topic 1 question 303 discussion	2023-07-08 16:19:00	/discussions/isaca/view/108648-exam-cisa-topic-1-question-303-discussion/

Exam CISM topic 1 question 312 discussion	2023-07-08 14:51:00	/discussions/isaca/view/106125-exam-cism-topic-1-question-312-discussion/

Exam CISM topic 1 question 203 discussion	2023-07-08 12:31:00	/discussions/isaca/view/98947-exam-cism-topic-1-question-203-discussion/

Exam CRISC topic 1 question 410 discussion	2023-07-08 05:53:00	/discussions/isaca/view/54147-exam-crisc-topic-1-question-410-discussion/

E

In [ ]:
               
    driver.quit()

In [ ]:
fname = 'DBS/DBS-Q000.html'

with open(fname, "r", encoding='utf-8') as file:
    html = file.read()
    file.close()
        
#print(html)
bs_template = BeautifulSoup(html, 'html.parser')

bs_template

<!DOCTYPE html>
<html lang="en">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<base href="https://changwhaj.github.io/assets/"/>
</head>
<body class="home2"><div style="position:relative;">
<!-- RubyOnRails - formstatic / kaminari -->
<meta content="authenticity_token" name="csrf-param"/>
<!-- Global site tag (gtag.js) - Google Analytics -->
<script async="" src="https://www.google-analytics.com/analytics.js" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-119892649-1"></script>
<script>
    window.dataLayer = window.dataLayer || [];
    function gtag() {
        dataLayer.push(arguments);
    }
    gtag('js', new Date());

    gtag('config', 'UA-119892649-1');
</script>
<!-- meta tag -->
<meta charset="utf-8"/>
<title>Exam AWS Certified Database - Specialty topic 1 question 250 discussion - ExamTopics</title>
<!-- responsive tag -->
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="

In [ ]:
print(str(bs_template).replace('</div>', '</DIV>'))

<!DOCTYPE html>
<html lang="en">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<base href="https://changwhaj.github.io/assets/"/>
</head>
<body class="home2"><div style="position:relative;">
<!-- RubyOnRails - formstatic / kaminari -->
<meta content="authenticity_token" name="csrf-param"/>
<!-- Global site tag (gtag.js) - Google Analytics -->
<script async="" src="https://www.google-analytics.com/analytics.js" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-119892649-1"></script>
<script>
    window.dataLayer = window.dataLayer || [];
    function gtag() {
        dataLayer.push(arguments);
    }
    gtag('js', new Date());

    gtag('config', 'UA-119892649-1');
</script>
<!-- meta tag -->
<meta charset="utf-8"/>
<title>Exam AWS Certified Database - Specialty topic 1 question 250 discussion - ExamTopics</title>
<!-- responsive tag -->
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="

In [ ]:
import math
QUESTIONS_PER_PAGE = 5
QUESTIONS_TOTAL = 258
PAGE_COUNTS = QUESTIONS_TOTAL / QUESTIONS_PER_PAGE
print(round(QUESTIONS_TOTAL / QUESTIONS_PER_PAGE + 0.5))
math.ceil(1.5)

52


2

In [ ]:
fname = 'DBS/DBS-Q000.html'

with open(fname, "r", encoding='utf-8') as file:
    html = file.read()
    file.close()
        
#print(html)
bs_template = BeautifulSoup(html, 'html.parser')

title = bs_template.find('title')
title.string = "TTTTT"
div_container = bs_template.find("div", {"class": "container"})
div_container.string = "XXXXXXXXXXXXXXXXXXXX"

# for div_container in bs_template.find_all("div", {"class": "container"}):
#     div_container.string = "XXXXX"

print(bs_template)

<!DOCTYPE html>

<html lang="en">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<base href="https://changwhaj.github.io/assets/"/>
</head>
<body class="home2"><div style="position:relative;">
<!-- RubyOnRails - formstatic / kaminari -->
<meta content="authenticity_token" name="csrf-param"/>
<!-- Global site tag (gtag.js) - Google Analytics -->
<script async="" src="https://www.google-analytics.com/analytics.js" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-119892649-1"></script>
<script>
    window.dataLayer = window.dataLayer || [];
    function gtag() {
        dataLayer.push(arguments);
    }
    gtag('js', new Date());

    gtag('config', 'UA-119892649-1');
</script>
<!-- meta tag -->
<meta charset="utf-8"/>
<title>TTTTT</title>
<!-- responsive tag -->
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<!-- favicon -->
<link 

In [ ]:
format(int("812"), '04')


'0812'

In [ ]:
    driver.quit()

In [ ]:
def open_exam(driver, pageno):
    exam_url = 'http://webcache.googleusercontent.com/search?q=cache:' + \
                'https://www.examtopics.com/exams/amazon/aws-certified-database-specialty/view/' + str(pageno) + '/'
    # discuss_url = 'https://www.examtopics.com/ajax/discussion/load-complete/?discussion-id=35981'
    page = driver.get(exam_url)
    time.sleep(randint(1, 10))
    return

def open_amazondiscuss(driver, pageno):
    discuss_url = \ # 'http://webcache.googleusercontent.com/search?q=cache:' + \
                'https://www.examtopics.com/discussions/amazon/' + str(pageno) + '/'
    # discuss_url = 'https://www.examtopics.com/ajax/discussion/load-complete/?discussion-id=35981'
    page = driver.get(discuss_url)
    time.sleep(randint(1, 10))
    return


def remove_element(driver, pageno):
    driver.execute_script("return document.getElementsByClassName('btn btn-success')[0].setAttribute('href','exam/aws/DBS/page/DBS-P" + str(pageno - 1) + ".html');")
    driver.execute_script("return document.getElementsByClassName('btn btn-success pull-right')[0].setAttribute('href','exam/aws/DBS/page/DBS-P" + str(pageno + 1) + ".html');")
    for i in range(4):
        try:
            driver.execute_script("return document.getElementsByClassName('btn btn-secondary question-discussion-button d-print-none')[" + str(i) + "].setAttribute('href','exam/aws/DBS/DBS-Q" + ('%03d' % ((pageno-1)*4 + i + 1)) + ".html');")
        except:
            continue

    driver.execute_script("""
        function getElementByXpath(path) {
            return document.evaluate(path, document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue;
        }
        function removeElement(element) {
            if (element !== undefined) {
                element.remove();
            }
        }
        function removeElemAttr(element, attr) {
            if (element !== undefined && element !== null) {
                element.removeAttribute(attr);
            }
        }
        removeElement(document.getElementById('bN015htcoyT__google-cache-hdr'));
        removeElement(getElementByXpath("/html/head/style"));
        removeElement(document.getElementsByClassName('full-width-header')[0]);
        removeElement(document.getElementsByClassName('exam-view-header')[0]);
        removeElement(document.getElementsByClassName('row')[0]);
        removeElement(document.getElementById('rs-footer'));
        getElementByXpath("/html/head/base").setAttribute('href','https://changwhaj.github.io/assets/');
        getElementByXpath("/html").removeAttribute("class");
        // document.getElementsByClassName('btn btn-success')[0].setAttribute('href','/exam/AWS/DBS/DBS-P1.html');
        // document.getElementsByClassName('btn btn-success pull-right')[0].setAttribute('href','/exam/AWS/DBS/DBS-P3.html');

        // removeElement(document.getElementsByClassName('page-navigation-bar')[0]);
        removeElement(document.getElementById('engagement-modal'));
        removeElement(document.getElementsByClassName('modal-backdrop show')[1]);
        removeElement(document.getElementsByClassName('modal-backdrop show')[0]);
        getElementByXpath("/html/body/base").removeAttribute("href");
        return;
    """)

def save_html(driver, fname):
    html = driver.page_source
    bs = BeautifulSoup(html, 'html.parser')
    print(fname)
    with open(fname, "w", encoding='utf-8') as file:
        file.write('<!DOCTYPE html>\n' + str(bs))


SyntaxError: unexpected character after line continuation character (Temp/ipykernel_20368/3573070062.py, line 10)

In [ ]:
if __name__ == "__main__":
    driver = webdriver.Chrome('c:/temp/chromedriver.exe')

    for i in range(47)[40:45]:
        pageno = i + 1
        fname = 'DBS/page/DBS-P' + str(pageno) + '.html'
        print(fname)

        my_file = Path(fname)
        if my_file.is_file():
            # file exists
            continue

        try:
            open_exam(driver, pageno)
            driver.switch_to.window(driver.window_handles[0])
            print(driver.title)
            if (driver.title == 'Error 404 (Not Found)!!1'):
                with open(fname, "w", encoding='utf-8') as file:
                    file.write('Error 404 (Not Found)')
                continue

            remove_element(driver, pageno)

        except:
            time.sleep(randint(1, 20))
            pass

        finally:
            save_html(driver, fname)
        
    driver.quit()

In [ ]:
    driver = webdriver.Chrome('c:/temp/chromedriver.exe')

In [ ]:
pageno = 1
fname = 'DBS/page/DBS-P' + str(pageno) + '.html'
my_file = Path(fname)
print(fname)
if my_file.is_file():
    # file exists
    print(fname)

DBS/page/DBS-P1.html


In [ ]:
open_exam(driver, 6)

In [ ]:
driver.switch_to.window(driver.window_handles[0])
print(driver.title)

AWS Certified Database - Specialty Exam – Free Exam Q&As, Page 5 | ExamTopics


In [ ]:
body > div.sec-spacer > div > div:nth-child(3) > div > div
    /html/body/div[2]/div/div[3]/div/div
container-fluid discussion-list

In [ ]:
driver.switch_to.window(driver.window_handles[0])
print(driver.title)
bs = BeautifulSoup(driver.page_source, 'html.parser')
div_card = bs.find_all("div", {"class": "container-fluid discussion-list"})
a = bs.find_all("a", {"class": "discussion-link"})
for i in range(len(a)):
    print(a[i]["href"])

Amazon discussions - ExamTopics Forum
/discussions/amazon/view/48843-exam-aws-certified-database-specialty-topic-1-question-140/
/discussions/amazon/view/46764-exam-aws-certified-solutions-architect-associate-saa-c02/
/discussions/amazon/view/68751-exam-aws-certified-developer-associate-topic-1-question-372/
/discussions/amazon/view/65738-exam-aws-certified-database-specialty-topic-1-question-157/
/discussions/amazon/view/51710-exam-aws-certified-data-analytics-specialty-topic-1-question/
/discussions/amazon/view/49205-exam-aws-certified-database-specialty-topic-1-question-104/
/discussions/amazon/view/35803-exam-aws-certified-solutions-architect-associate-saa-c02/
/discussions/amazon/view/27770-exam-aws-certified-solutions-architect-associate-saa-c02/
/discussions/amazon/view/49007-exam-aws-certified-database-specialty-topic-1-question-126/
/discussions/amazon/view/65925-exam-aws-certified-database-specialty-topic-1-question-162/
/discussions/amazon/view/74242-exam-aws-certified-data-

In [ ]:
    driver = webdriver.Chrome('c:/temp/chromedriver.exe')
    
    fname = 'AmazonDiscussList.html'
    print(fname)

    my_file = Path(fname)


AmazonDiscussList.html


In [ ]:
def open_amazondiscuss(driver, pageno):
    discuss_url = 'https://www.examtopics.com/discussions/amazon/' + str(pageno) + '/'
    # discuss_url = 'https://www.examtopics.com/ajax/discussion/load-complete/?discussion-id=35981'
    page = driver.get(discuss_url)
    time.sleep(randint(1, 10))
    return


In [ ]:
#pageno = 2
#open_amazondiscuss(driver, pageno)
driver.switch_to.window(driver.window_handles[0])
print(driver.title + "-" + str(pageno))
bs = BeautifulSoup(driver.page_source, 'html.parser')
a = bs.find_all("a", {"class": "btn btn-secondary question-discussion-button d-print-none"})
for i in range(len(a)):
    print(a[i]["href"])

AWS Certified Cloud Practitioner Exam – Free Exam Q&As, Page 37 | ExamTopics-375
/discussions/amazon/view/20816-exam-aws-certified-cloud-practitioner-topic-1-question-166/
/discussions/amazon/view/6955-exam-aws-certified-cloud-practitioner-topic-1-question-42/
/discussions/amazon/view/44218-exam-aws-certified-cloud-practitioner-topic-1-question-526/
#
#
/discussions/amazon/view/17028-exam-aws-certified-cloud-practitioner-topic-1-question-116/
/discussions/amazon/view/44217-exam-aws-certified-cloud-practitioner-topic-1-question-527/
/discussions/amazon/view/26020-exam-aws-certified-cloud-practitioner-topic-1-question-240/
/discussions/amazon/view/35036-exam-aws-certified-cloud-practitioner-topic-1-question-486/
/discussions/amazon/view/33445-exam-aws-certified-cloud-practitioner-topic-1-question-464/


In [ ]:
a = bs.find_all("a", {"class": "discussion-link"})
a[0].text.strip()

'Exam AWS Certified Solutions Architect - Associate SAA-C02 topic 1 question 791 discussion'

In [ ]:
with open("AmazonDiscussList.txt", "w", encoding='utf-8') as file:    
    for i in range(375):
        pageno = i + 1

        open_amazondiscuss(driver, pageno)

        driver.switch_to.window(driver.window_handles[0])
        print(driver.title + "-" + str(pageno))
        bs = BeautifulSoup(driver.page_source, 'html.parser')
        div_card = bs.find_all("div", {"class": "container-fluid discussion-list"})
        a = bs.find_all("a", {"class": "discussion-link"})
        for i in range(len(a)):
            file.write(str(a[i].text.strip())+"\t")
            file.write(str(a[i]["href"])+"\n")
            
file.close()

Amazon discussions - ExamTopics Forum-1
Amazon discussions - ExamTopics Forum-2
Amazon discussions - ExamTopics Forum-3
Amazon discussions - ExamTopics Forum-4
Amazon discussions - ExamTopics Forum-5
Amazon discussions - ExamTopics Forum-6
Amazon discussions - ExamTopics Forum-7
Amazon discussions - ExamTopics Forum-8
Amazon discussions - ExamTopics Forum-9
Amazon discussions - ExamTopics Forum-10
Amazon discussions - ExamTopics Forum-11
Amazon discussions - ExamTopics Forum-12
Amazon discussions - ExamTopics Forum-13
Amazon discussions - ExamTopics Forum-14
Amazon discussions - ExamTopics Forum-15
Amazon discussions - ExamTopics Forum-16
Amazon discussions - ExamTopics Forum-17
Amazon discussions - ExamTopics Forum-18
Amazon discussions - ExamTopics Forum-19
Amazon discussions - ExamTopics Forum-20
Amazon discussions - ExamTopics Forum-21
Amazon discussions - ExamTopics Forum-22
Amazon discussions - ExamTopics Forum-23
Amazon discussions - ExamTopics Forum-24
Amazon discussions - Exam

Amazon discussions - ExamTopics Forum-199
Amazon discussions - ExamTopics Forum-200
Amazon discussions - ExamTopics Forum-201
Amazon discussions - ExamTopics Forum-202
Amazon discussions - ExamTopics Forum-203
Amazon discussions - ExamTopics Forum-204
Amazon discussions - ExamTopics Forum-205
Amazon discussions - ExamTopics Forum-206
Amazon discussions - ExamTopics Forum-207
Amazon discussions - ExamTopics Forum-208
Amazon discussions - ExamTopics Forum-209
Amazon discussions - ExamTopics Forum-210
Amazon discussions - ExamTopics Forum-211
Amazon discussions - ExamTopics Forum-212
Amazon discussions - ExamTopics Forum-213
Amazon discussions - ExamTopics Forum-214
Amazon discussions - ExamTopics Forum-215
Amazon discussions - ExamTopics Forum-216
Amazon discussions - ExamTopics Forum-217
Amazon discussions - ExamTopics Forum-218
Amazon discussions - ExamTopics Forum-219
Amazon discussions - ExamTopics Forum-220
Amazon discussions - ExamTopics Forum-221
Amazon discussions - ExamTopics Fo

In [ ]:
driver.switch_to.window(driver.window_handles[0])
print(driver.title)

bs = BeautifulSoup(driver.page_source, 'html.parser')
div_questions = bs.find_all("div", {"class": "questions-container"})

with open("DBS_Page.html", "a", encoding='utf-8') as file:
    #file.append(div_questions)
    file.write(str(div_questions))

AWS Certified Database - Specialty Exam – Free Exam Q&As, Page 41 | ExamTopics


In [ ]:
    driver.execute_script("return document.getElementsByClassName('btn btn-success')[0].setAttribute('href','exam/aws/DBS/page/DBS-P" + str(pageno - 1) + ".html');")
    driver.execute_script("return document.getElementsByClassName('btn btn-success pull-right')[0].setAttribute('href','exam/aws/DBS/page/DBS-P" + str(pageno + 1) + ".html');")

In [1]:
import pyautogui

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup

#from fake_useragent import UserAgent
from random import randrange
from datetime import datetime
import time

def set_chrome_driver():
    service = Service(executable_path=r'c:/temp/chromedriver.exe')
    userAgent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)s Chrome/92.0.4515.131 Safari/537.36"
    options = webdriver.ChromeOptions()
    options.add_argument(f"user-agent={userAgent}")
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    options.add_argument('--no-sandbox')
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--lang=kr')  # set your language here
    driver = webdriver.Chrome(service=service, options=options)

    driver.set_window_size(1400, 1040)
    
    return driver

def set_translate_to_kr(driver):
    actionChains = ActionChains(driver)
    actionChains.context_click().perform()

    # here maybe problem. Debug it:
    for i in range(3):
        pyautogui.press('up')

    pyautogui.press('enter')
    pyautogui.sleep(1)

def scroll_page(driver):
    # Set the interval between scrolls in seconds
    scroll_interval = 1

    # Get the initial height of the page
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Send the down arrow key to scroll down the page
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
        
        # Wait for the specified interval
        time.sleep(scroll_interval)
        
        # Calculate the new height of the page after scrolling
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        # Check if the page has reached the end
        if new_height == last_height:
            break
        
        # Update the previous height
        last_height = new_height

def save_page(driver, fname):
    bs = BeautifulSoup(driver.page_source, 'html.parser')

    with open(fname, "w", encoding='utf-8') as file:
        file.write('<!DOCTYPE html>\n' + str(bs))
        # file.write(bs.prettify())

In [2]:
driver = set_chrome_driver()

for i in range(583)[35:37]:

    url = f'file:///C:/Users/Changwha/MyProjects/assets/assets/exam/aws/SAA_C03/SAA-Q{i+1:04}.html'
    fn = f'SAA_C03/kr/SAA-Q{i+1:04}-KR.html'
    try:
        driver.get(url)
        driver.switch_to.window(driver.window_handles[0])

        driver.find_element(By.CSS_SELECTOR, 'a.btn.btn-primary.reveal-solution').click()
        set_translate_to_kr(driver)
        driver.find_element(By.CSS_SELECTOR, 'a.btn.btn-primary.hide-solution').click()
        scroll_page(driver)
        driver.find_element(By.CSS_SELECTOR, '#scrollUp').click()
        time.sleep(1)
        driver.execute_script("arguments[0].remove();", driver.find_element(By.CSS_SELECTOR, '#goog-gt-tt'))
        driver.execute_script("arguments[0].remove();", driver.find_element(By.CSS_SELECTOR, 'head > link'))
        driver.execute_script("arguments[0].removeAttribute('class');", driver.find_element(By.TAG_NAME, 'html'))
        save_page(driver, fn)

    except Exception as e:
        print("Error:", str(e))
        print(f"*** Make question error !!! {e}")
        pass

driver.close()

In [95]:
driver.find_element(By.CSS_SELECTOR, 'a.btn.btn-primary.hide-solution').click()


In [96]:
driver.close()
# or webdriver.quit()